# Create datasets

Use pre-prepared image downloads to create datasets with train/test splits for model

## Config

In [4]:
import pandas as pd
import sqlalchemy as sqa
from sklearn.model_selection import train_test_split
import json
from typing import Dict, Union

In [ ]:
# PARAMETERS
# Database parameters
db_container = "metadata_db"
db_user = "pguser"
db_password = "pgpassword"
db_port = 5432
db_database = "metadata"
db_prefix = "postgresql"
metadata_tbl = "base_images"
datasets_tbl = "datasets"
dataset_img_tbl = "dataset_images"


# Dataset parameters
datasets = [
    # TODO
]

In [ ]:
class Image_Dataset_Builder():
    """
    Class for creating a dataset from a metadata table and configs
    """
    df_img: Union(pd.DataFrame, None) = None
    def __init__(self):
        pass
    
    
    # TODO
    # Clear data associated with this dataset from dataset
    def clear_db_data(self):
        pass
    
    # TODO
    # Create (if not exists) or clear (if exists) data from target directory
    def clear_target_dir(self):
        pass
    
    # TODO
    # Use config to create dataframe of desired image outputs
    def create_image_df(self):
        pass
    
    # TODO
    # Copy files from image dataframe from source to target directory
    def copy_image_files(self):
        pass
    
    # TODO
    # Store associated data to the metadata db (dataset table and dataset images table)
    def store_db_metadata(self):
        pass
    
    
    # TODO
    # Builds the dataset by running the other methods
    def build_dataset(self):
        pass